# Scraping de Lapypipa

Nuestro objetivo consiste en escrapear la pagina de vestidos de Ladypipa para ello usaremos tanto Selenium como Beautiful Soup.

In [1]:
import sys
import os
import pandas as pd
import json

sys.path.append(os.path.abspath('../'))
from src import funciones_scraping as fs

Con Selenium accedemos a la página principal de vestidos y sacamos todos los links a los que posterioirmente vamos a acceder. Estos links los almacenamos dentro de la carpeta datos para tenerlos guardados y solo tener que ejecutar una vez la función.

In [6]:
links_ladypipa = fs.obtencion_links_ladypipa()

with open("../datos/scraping/links_ladypipa.json", "w") as archivo_json:
    json.dump(links_ladypipa, archivo_json)

A continuación mediande BeautifulSoup iremos recabando de cada vestido la información necesaria para crear el dataframe.

In [2]:
with open("../datos/scraping/links_ladypipa.json", "r") as archivo_json:
    links_ladypipa = json.load(archivo_json)

In [3]:
dic_vestido = fs.extraccion_info_ladypipa(links_ladypipa)

In [6]:
df_lady_pipa = pd.DataFrame(dic_vestido)
df_lady_pipa.drop_duplicates(inplace=True)

Una vez hemos comprobado que todos los campos son correctos la almacenamos.

In [8]:
df_lady_pipa.to_csv("../datos/dataframes/df_ladypipa.csv")

In [9]:
df_lady_pipa.head(3)

,nombre,marca,precio,color,talla,categoria
0,Vestido Salima,Ladypipa,"129,95",Nenúfar caldera,XXS,midi
1,Vestido Salima,Ladypipa,"129,95",Nenúfar caldera,XS,midi
2,Vestido Salima,Ladypipa,"129,95",Nenúfar caldera,S,midi


_____________

In [31]:
url_base="https://ladypipa.com"

dic_vestido = {
        "nombre" : [],
        "marca" : [],
        "precio" : [],
        "color" : [],
        "talla" : [],
        "categoria" :[]
        }

for url in links_ladypipa:
    res = requests.get(url)
    if res.status_code != 200:
        print(url)

    sopa = BeautifulSoup(res.content, "html.parser")


    contenedor_enlaces= sopa.find("div", {"class":"variant-picker__option-values wrap gap-2"}).find_all('a', class_='color-swatch')
    enlaces_colores = [enlace['href'] for enlace in contenedor_enlaces]

    for enlace in enlaces_colores:

        url_aux = url_base + enlace
        res = requests.get(url_aux)
        if res.status_code != 200:
            print(url)
        
        
        sopa = BeautifulSoup(res.content, "html.parser")

        nombre = sopa.find("div", {"class":"product-info__title"}).text.replace("\n","").strip()
        marca = "Ladypipa"

        # Extraemos el color
        contenedor_color = sopa.find("div", {"class":"variant-picker__option-info"}).text
        color = contenedor_color.strip().replace("Color:", "").strip()

        # Extraemos el precio
        patron = r"(\d+(?:,\d{1,2})?)\s*€"
        resultado = re.search(patron, sopa.find("sale-price").text)
        precio = resultado.group(1)

        # Buscamos en la descripcion cómo es el vestido para asignarle una categoría
        desc = sopa.select("div.prose p")[2].text

        patron = r'\b(largo|midi|corto|maxi|mini)\b'
        coincidencias = re.findall(patron, desc)
        categoria = coincidencias[0]

        # Sacamos las tallas disponibles
        lista_tallas=[]
        contenedor_tallas = sopa.find("div", {"class":"form-control"}).find("select").find_all('option')
        for talla in contenedor_tallas:
            if not talla.has_attr('disabled'):
                lista_tallas.append(talla.text.split()[0])
        lista_tallas

        for elem in lista_tallas:
            dic_vestido["nombre"].append(nombre)
            dic_vestido["marca"].append(marca)
            dic_vestido["precio"].append(precio)
            dic_vestido["color"].append(color)
            dic_vestido["talla"].append(elem)
            dic_vestido["categoria"].append(categoria)

Nenúfar caldera
Nenúfar negro
Manila verde
Marino
Marino
Aceite
Melón
Sakura
Morado
Negro
Caldera
Marfil
Salvia
Coral
Marfil
Marino
Rojo
Irapa teja
Nantai azul
Nantai burdeos
Azul
Rojo
Nantai azul
Nantai burdeos
Arrecife verde
Uva
Celadón
Rojo
Caldera
Celadón
Marino
Burdeos
Cobre
Caldera
Azul
Rojo
Manila verde
Marino
Arrecife verde
Uva
Marfil
Marino
Rojo
Caldera
Celadón
Marino
Marfil
Caldera
Marino
Irapa lavanda
Perseo
Verde
Nenúfar caldera
Marfil
Caldera
Marino
Caldera
Celadón
Marino
Morado
Chocolate
Caldera
Azul marino
Rojo
Bogotá berenjena
Manila
Irapa rojo
Coyoacán mineral
Rosé
Blanco
Cereza
Marfil
Salvia
Coral
Blanco
Marfil
Marino
Rojo
Aceite


In [ ]:
df = pd.DataFrame(dic_vestido)
df.shape

In [36]:
df[df.duplicated()]

,nombre,marca,precio,color,talla,categoria
142,Vestido Fuji Nantai marino,Ladypipa,"179,95",Nantai azul,XXS,midi
143,Vestido Fuji Nantai marino,Ladypipa,"179,95",Nantai azul,XS,midi
144,Vestido Fuji Nantai marino,Ladypipa,"179,95",Nantai azul,S,midi
145,Vestido Fuji Nantai marino,Ladypipa,"179,95",Nantai azul,M,midi
146,Vestido Fuji Nantai marino,Ladypipa,"179,95",Nantai azul,L,midi
...,...,...,...,...,...,...
493,Vestido Idina,Ladypipa,"109,95",Rojo,XS,largo
494,Vestido Idina,Ladypipa,"109,95",Rojo,S,largo
495,Vestido Idina,Ladypipa,"109,95",Rojo,M,largo
496,Vestido Idina,Ladypipa,"109,95",Rojo,L,largo


In [39]:
df[df["nombre"]=="Vestido Fuji"]

,nombre,marca,precio,color,talla,categoria
120,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,XXS,midi
121,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,XS,midi
122,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,S,midi
123,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,M,midi
124,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,L,midi
125,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,XL,midi
126,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,3XL,midi
127,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,4XL,midi
148,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,XXS,midi
149,Vestido Fuji,Ladypipa,"179,95",Nantai burdeos,XS,midi
